In [1]:
import os

import torch
from torch import nn
import bitsandbytes as bnb
from datasets import load_dataset
import transformers as T
from transformers import AutoTokenizer, AutoConfig
from transformers import LlamaForCausalLM, LlamaTokenizer
from peft import prepare_model_for_int8_training, LoraConfig, get_peft_model, get_peft_model_state_dict

os.environ["CUDA_VISIBLE_DEVICES"] = "0"


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/opt/aws-ofi-nccl/lib:/usr/local/lib:/usr/lib: did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//127.0.0.1'), PosixPath('56529'), PosixPath('pmix-server.2040307;tcp4')}
  warn(msg)
/fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('1'), PosixPath('/usr/share/modules/$MODULE_VERSION/modulefiles')}
  warn(msg)
/fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:

## 2023-05-13

In [14]:
DATA_FILES = "/fsx/jp-llm/instruction_tuning/datasets/japanese_alpaca_data.json"
VAL_SET_SIZE = 2000
CUTOFF_LEN = 256

In [5]:
data = load_dataset("json", data_files=DATA_FILES)
train_val = data["train"].train_test_split(
    test_size=VAL_SET_SIZE, shuffle=True, seed=42
)
train_data = train_val["train"]
val_data = train_val["test"]

Found cached dataset json (/admin/home-fujiki/.cache/huggingface/datasets/json/default-a28b5bb2c6979ed7/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached split indices for dataset at /admin/home-fujiki/.cache/huggingface/datasets/json/default-a28b5bb2c6979ed7/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-547705db6822a05d.arrow and /admin/home-fujiki/.cache/huggingface/datasets/json/default-a28b5bb2c6979ed7/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4/cache-3c493914ff88bf93.arrow


In [6]:
def generate_prompt(data_point):
        # sorry about the formatting disaster gotta move fast
        if data_point["input"]:
            return f"""以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。
### 命令:
{data_point["instruction"]}
### 入力:
{data_point["input"]}
### 応答:
{data_point["output"]}"""
        else:
            return f"""以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。
### 命令:
{data_point["instruction"]}
### 応答:
{data_point["output"]}"""

In [8]:
val_data[0]

{'output': '「関節炎の人にとって、最適な運動はヨガ、水泳、またはウォーキングなどの低負荷の運動です。これらの運動は、関節炎の症状を悪化させることなく、運動の利点を提供します。」',
 'instruction': '「関節炎を持つ人にとって最適な運動は何ですか？」',
 'input': ''}

In [10]:
val_data_prompt = val_data.map(lambda x: generate_prompt(x))

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2988452/2741065921.py:1 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2988452/2741065921.py'               │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/datasets/arrow_dataset.py:578 │
│ in wrapper                                                                                │
│                                                                                           │
│    575 │   │   else:                                                                      │
│    576 │   │   │   self: "Dataset" = kwargs.pop("self")                                   │
│    577 │   │   # apply actual function                                                    │
│ ❱  578 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)         │
│    579 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) el │
│    580 │   │   for dataset in datasets:                                                   │
│    581 │   │   │   # Remove task templates if a column mapping of the template is no long │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/datasets/arrow_dataset.py:543 │
│ in wrapper                                                                                │
│                                                                                           │
│    540 │   │   │   "output_all_columns": self._output_all_columns,                        │
│    541 │   │   }                                                                          │
│    542 │   │   # apply actual function                                                    │
│ ❱  543 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)         │
│    544 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) el │
│    545 │   │   # re-apply format to the output                                            │
│    546 │   │   for dataset in datasets:                                                   │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/datasets/arrow_dataset.py:307 │
│ 3 in map                                                                                  │
│                                                                                           │
│   3070 │   │   │   │   │   leave=False,                                                   │
│   3071 │   │   │   │   │   desc=desc or "Map",                                            │
│   3072 │   │   │   │   ) as pbar:                                                         │
│ ❱ 3073 │   │   │   │   │   for rank, done, content in Dataset._map_single(**dataset_kwarg │
│   3074 │   │   │   │   │   │   if done:                                                   │
│   3075 │   │   │   │   │   │   │   shards_done += 1                                       │
│   3076 │   │   │   │   │   │   │   logger.debug(f"Finished processing shard number {rank} │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/datasets/arrow_dataset.py:342 │
│ 7 in _map_single                                                                          │
│                                                                                           │
│   3424 │   │   │   │   if not batched:                                                    │
│   3425 │   │   │   │   │   _time = time.time()    

In [11]:
generate_prompt(val_data[0])

'以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。\n### 命令:\n「関節炎を持つ人にとって最適な運動は何ですか？」\n### 応答:\n「関節炎の人にとって、最適な運動はヨガ、水泳、またはウォーキングなどの低負荷の運動です。これらの運動は、関節炎の症状を悪化させることなく、運動の利点を提供します。」'

In [12]:
print(generate_prompt(val_data[0]))

以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。
### 命令:
「関節炎を持つ人にとって最適な運動は何ですか？」
### 応答:
「関節炎の人にとって、最適な運動はヨガ、水泳、またはウォーキングなどの低負荷の運動です。これらの運動は、関節炎の症状を悪化させることなく、運動の利点を提供します。」


In [10]:
val_data_token = val_data.map(lambda x: tokenize(generate_prompt(x)))

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2988452/2741065921.py:1 in <module>                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2988452/2741065921.py'               │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/datasets/arrow_dataset.py:578 │
│ in wrapper                                                                                │
│                                                                                           │
│    575 │   │   else:                                                                      │
│    576 │   │   │   self: "Dataset" = kwargs.pop("self")                                   │
│    577 │   │   # apply actual function                                                    │
│ ❱  578 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)         │
│    579 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) el │
│    580 │   │   for dataset in datasets:                                                   │
│    581 │   │   │   # Remove task templates if a column mapping of the template is no long │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/datasets/arrow_dataset.py:543 │
│ in wrapper                                                                                │
│                                                                                           │
│    540 │   │   │   "output_all_columns": self._output_all_columns,                        │
│    541 │   │   }                                                                          │
│    542 │   │   # apply actual function                                                    │
│ ❱  543 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)         │
│    544 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) el │
│    545 │   │   # re-apply format to the output                                            │
│    546 │   │   for dataset in datasets:                                                   │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/datasets/arrow_dataset.py:307 │
│ 3 in map                                                                                  │
│                                                                                           │
│   3070 │   │   │   │   │   leave=False,                                                   │
│   3071 │   │   │   │   │   desc=desc or "Map",                                            │
│   3072 │   │   │   │   ) as pbar:                                                         │
│ ❱ 3073 │   │   │   │   │   for rank, done, content in Dataset._map_single(**dataset_kwarg │
│   3074 │   │   │   │   │   │   if done:                                                   │
│   3075 │   │   │   │   │   │   │   shards_done += 1                                       │
│   3076 │   │   │   │   │   │   │   logger.debug(f"Finished processing shard number {rank} │
│                                                                                           │
│ /fsx/home-fujiki/venvs/polyglot/lib/python3.8/site-packages/datasets/arrow_dataset.py:342 │
│ 7 in _map_single                                                                          │
│                                                                                           │
│   3424 │   │   │   │   if not batched:                                                    │
│   3425 │   │   │   │   │   _time = time.time()    

In [15]:
tokenize(generate_prompt(val_data[0]))

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2988452/363386410.py:1 in <module>                                         │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2988452/363386410.py'                │
│ /tmp/ipykernel_2988452/4250418532.py:4 in tokenize                                        │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2988452/4250418532.py'               │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'tokenizer' is not defined

In [16]:
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-1b", use_fast=False)  # , add_eos_token=True)

In [17]:
tokenizer("日本の首都はどこですか？")

{'input_ids': [3174, 4771, 18507, 1499, 36631, 3], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [18]:
tokenizer(generate_prompt(val_data[0]))

{'input_ids': [23006, 36372, 18356, 36384, 13283, 4187, 36379, 36372, 14571, 164, 5822, 16170, 36503, 2526, 4830, 16, 36375, 3765, 36384, 16560, 3535, 28480, 16869, 12575, 7317, 36375, 5670, 37700, 37700, 36370, 4187, 36489, 87, 7864, 37732, 871, 26484, 9036, 1172, 5604, 1499, 1123, 5670, 37700, 37700, 36370, 16869, 36489, 87, 7864, 37732, 319, 898, 36372, 9036, 28270, 9247, 36372, 18136, 36372, 908, 15580, 309, 37119, 9965, 36371, 1172, 16, 36375, 2962, 28270, 36372, 7864, 16128, 28668, 8405, 647, 5890, 36372, 25333, 36637, 2431, 692, 97, 1405, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [19]:
_tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-1b", use_fast=False, add_eos_token=True)

def tokenize(prompt):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = _tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN + 1,
        padding="max_length",
    )
    return {
        "input_ids": result["input_ids"][:-1],
        "attention_mask": result["attention_mask"][:-1],
    }

In [20]:
tokenize(generate_prompt(val_data[0]))

{'input_ids': [23006,
  36372,
  18356,
  36384,
  13283,
  4187,
  36379,
  36372,
  14571,
  164,
  5822,
  16170,
  36503,
  2526,
  4830,
  16,
  36375,
  3765,
  36384,
  16560,
  3535,
  28480,
  16869,
  12575,
  7317,
  36375,
  5670,
  37700,
  37700,
  36370,
  4187,
  36489,
  87,
  7864,
  37732,
  871,
  26484,
  9036,
  1172,
  5604,
  1499,
  1123,
  5670,
  37700,
  37700,
  36370,
  16869,
  36489,
  87,
  7864,
  37732,
  319,
  898,
  36372,
  9036,
  28270,
  9247,
  36372,
  18136,
  36372,
  908,
  15580,
  309,
  37119,
  9965,
  36371,
  1172,
  16,
  36375,
  2962,
  28270,
  36372,
  7864,
  16128,
  28668,
  8405,
  647,
  5890,
  36372,
  25333,
  36637,
  2431,
  692,
  97,
  1405,
  3,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,
  4,


## Evaluation

In [2]:
MICRO_BATCH_SIZE = 4
BATCH_SIZE = 128
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
EPOCHS = 3 
LEARNING_RATE = 3e-4
CUTOFF_LEN = 256
LORA_R = 8
LORA_ALPHA = 16
LORA_DROPOUT = 0.05
VAL_SET_SIZE = 2000

In [15]:
torch.cuda.is_available()

True

In [8]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig

In [38]:
def load_model(model_dir):
    tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-1b", use_fast=False, add_eos_token=True)
    # config = AutoConfig.from_pretrained(model_dir)
    # config.is_decoder = True
    model = AutoModelForCausalLM.from_pretrained(
        "rinna/japanese-gpt-1b",
        load_in_8bit=True,
        device_map="auto",
    )
    # if torch.cuda.is_available():
    #     model = model.to("cuda:0")
    return model, tokenizer

In [10]:
model, tokenizer = load_model("rinna/japanese-gpt-1b")

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [11]:
model_custom = PeftModel.from_pretrained(
    model, 
    "/fsx/jp-llm/instruction_tuning/outputs/train_002_rinna-1b_ja-alpaca-data/lora",
)

In [51]:
def generate_instruction_prompt(instruction, _input=None):
    if _input:
        return f"""以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 命令:
{instruction}

### 入力:
{_input}

### 応答:"""
    else:
        return f"""以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
{instruction}

### 応答:"""

In [25]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

In [26]:
gen_output = evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
---- Response -----
</s></s>


In [30]:
generation_config = GenerationConfig(
    # temperature=0.1,
    top_p=0.95,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [32]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=384
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [33]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s>タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 

In [45]:
tokenizer.pad_token_id = 0

generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [40]:
tokenizer.pad_token_id

0

In [42]:
tokenizer.decode(0), tokenizer.decode(4)

('[CLS]', '[PAD]')

In [46]:
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-1b", use_fast=False)

In [47]:
generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [48]:
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [53]:
tokenizer = AutoTokenizer.from_pretrained("rinna/japanese-gpt-1b", use_fast=False)
model_cudstom = AutoModelForCausalLM.from_pretrained(
    "rinna/japanese-gpt-1b",
    load_in_8bit=True,
    device_map="auto",
)
model_original = AutoModelForCausalLM.from_pretrained(
    "rinna/japanese-gpt-1b",
    load_in_8bit=True,
    device_map="auto",
)
model_custom = PeftModel.from_pretrained(
    model_custom, 
    "/fsx/jp-llm/instruction_tuning/outputs/train_002_rinna-1b_ja-alpaca-data/lora",
)
# model_original = PeftModel.from_pretrained(
#     model_original, 
#     "rinna/japanese-gpt-1b",
# )

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


In [58]:
print(tokenizer.cls_token_id, tokenizer.decode(tokenizer.cls_token_id))
print(tokenizer.bos_token_id, tokenizer.decode(tokenizer.bos_token_id))
print(tokenizer.eos_token_id, tokenizer.decode(tokenizer.eos_token_id))
print(tokenizer.pad_token_id, tokenizer.decode(tokenizer.pad_token_id))

0 [CLS]
2 <s>
3 </s>
4 [PAD]


In [62]:
model_custom.config.pad_token_id = tokenizer.pad_token_id = 0

In [63]:
generation_config = GenerationConfig(
    temperature=0.9,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [64]:
prompt = generate_instruction_prompt("日本の首都はどこですか？")

In [65]:
tokenizer(prompt)

{'input_ids': [23006, 36372, 18356, 36384, 13283, 4187, 36379, 36372, 14571, 164, 5822, 16170, 36503, 2526, 4830, 16, 36375, 3765, 36384, 16560, 3535, 28480, 16869, 12575, 7317, 36375, 5670, 37700, 37700, 36370, 3498, 36489, 3174, 4771, 18507, 1499, 36631, 5670, 37700, 37700, 36370, 16869, 36489, 3], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [66]:
model_custom.config.pad_token_id = tokenizer.pad_token_id = 0

generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=16,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [67]:
model_custom.config.pad_token_id = tokenizer.pad_token_id = 0

generation_config = GenerationConfig(
    temperature=0.9,
    top_p=0.75,
    num_beams=16,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [68]:
model_custom.config.pad_token_id = tokenizer.pad_token_id = 0

generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=512,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s>タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 ### タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。 

In [69]:
model_custom.config.pad_token_id = tokenizer.pad_token_id = 0

generation_config = GenerationConfig(
    temperature=0.1,
    top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=384,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [70]:
model_custom.config.pad_token_id = tokenizer.pad_token_id = 0

generation_config = GenerationConfig(
    temperature=0.1,
    # top_p=0.75,
    num_beams=4,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [71]:
model_custom.config.pad_token_id = tokenizer.pad_token_id = 0

generation_config = GenerationConfig(
    temperature=0.1,
    # top_p=0.75,
    num_beams=16,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>


In [72]:
model_custom.config.pad_token_id = tokenizer.pad_token_id = 0

generation_config = GenerationConfig(
    temperature=0.1,
    # top_p=0.75,
    num_beams=128,
)
def evaluate(model_aaa, instruction, input=None):
    prompt = generate_instruction_prompt(instruction, input)
    print(f"Prompt: ", prompt)
    print("-----")
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].cuda()
    print(f"input_ids.shape = {input_ids.shape}")
    generation_output = model_aaa.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )

    output = tokenizer.decode(generation_output.sequences[0])
    # print("Output: ", output)
    # print("-----")
    response = output.split("### 応答:")[1].strip()
    if "命令" in response:
        response = response.split("### 命令")[0].strip()
    print("---- Response -----")
    print(response)

evaluate(model_custom,"日本の首都はどこですか？")

Prompt:  以下は、タスクを説明する命令と、さらなるコンテキストを提供する入力の組み合わせです。要求を適切に満たすような応答を書きなさい。

### 入力:
日本の首都はどこですか？

### 応答:
-----
input_ids.shape = torch.Size([1, 44])
---- Response -----
</s></s>
